In [1]:
import pickle, copy
import numpy as np
import pandas as pd
from PLANBERT.PLANBERT import PLANBERT

train_csv = pd.read_csv('example/example_train.csv')
valid_csv = pd.read_csv('example/example_valid.csv')
test_csv = pd.read_csv('example/example_train.csv')

Using TensorFlow backend.


In [2]:
train_csv

,user,t,item,feat_0,feat_1
0,5416,0,1798,306,0
1,5416,0,1798,306,0
2,5416,0,1798,306,0
3,5416,1,4282,1012,1
4,5416,2,6731,2170,32
...,...,...,...,...,...
153676,5525,1,4915,4512,58
153677,5525,2,7906,788,5
153678,5525,3,9029,1298,58
153679,5525,4,3491,123,0


In [3]:
planbert = PLANBERT(25, 10000, [5507, 69], 0, checkpoint='example/example_ckpt.h5')

In [4]:
predict = planbert.predict(test_csv, 'time', 9)

100%|█████████████████████| 430/430 [00:34<00:00, 12.30it/s]


In [5]:
planbert.test(test_csv, h_list=[9], r_list=[3], pool_size=25)

100%|█████████████████████| 430/430 [01:22<00:00,  5.20it/s]


({9: {3: [0.38691309623821524,
    array([0.35719557, 0.34458399, 0.36605657, 0.38431373, 0.36393443,
           0.34946871, 0.39583333, 0.35068913, 0.4086629 , 0.42572062,
           0.46236559, 0.42597403, 0.45993031, 0.5204918 , 0.50993377,
           0.5320197 ])]}},
 {9: {3: [0.30831699803976476,
    array([0.25387454, 0.25353218, 0.26206323, 0.28529412, 0.29289617,
           0.27744982, 0.29315476, 0.2863706 , 0.35404896, 0.37028825,
           0.38978495, 0.38441558, 0.41114983, 0.50409836, 0.50331126,
           0.50246305])]}})

In [ ]:
planbert = PLANBERT(25, 10000, [5507, 69], 0)
planbert.fit(train_csv, valid_csv)
planbert.test(test_csv, h_list=[9], r_list=[3], pool_size=25)